sentiment analysis

https://spraakbanken.gu.se/resurser/absabank-imm

In [ ]:
#!pip install scikit-learn==1.0.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.linear_model import SGDClassifier



In [ ]:
#@title
#!wget https://svn.spraakdata.gu.se/sb-arkiv/pub/imm_absabank/datasplit.zip

In [ ]:
!unzip /content/drive/MyDrive/datasplit_sent.zip

Archive:  /content/drive/MyDrive/datasplit_sent.zip
   creating: split10_consecutive_average/
  inflating: split10_consecutive_average/dev00.tsv  
  inflating: split10_consecutive_average/dev01.tsv  
  inflating: split10_consecutive_average/dev02.tsv  
  inflating: split10_consecutive_average/dev03.tsv  
  inflating: split10_consecutive_average/dev04.tsv  
  inflating: split10_consecutive_average/dev05.tsv  
  inflating: split10_consecutive_average/dev06.tsv  
  inflating: split10_consecutive_average/dev07.tsv  
  inflating: split10_consecutive_average/dev08.tsv  
  inflating: split10_consecutive_average/dev09.tsv  
  inflating: split10_consecutive_average/test00.tsv  
  inflating: split10_consecutive_average/test01.tsv  
  inflating: split10_consecutive_average/test02.tsv  
  inflating: split10_consecutive_average/test03.tsv  
  inflating: split10_consecutive_average/test04.tsv  
  inflating: split10_consecutive_average/test05.tsv  
  inflating: split10_consecutive_average/test06.tsv 

In [ ]:
#@title
import os
import pandas as pd

def readDir(pathToDir, split = "train"):
    listOfFiles = []
    for filPath in os.listdir(pathToDir):

        if split in filPath:
          if filPath.endswith(".tsv"):
              listOfFiles.append(os.path.join(pathToDir,filPath))
    return listOfFiles


In [ ]:
list_of_files = readDir("/content/split10_consecutive_average")

In [ ]:
#@title
df_list = []

for item in list_of_files:
    df_list.append(pd.read_csv( item, delimiter = "\t"))

train_df = pd.concat(df_list,ignore_index=False)


In [ ]:
train_df.sample(3)

,doc,par,text,label
2345,z01229_flashback-2669341,2,Jaha då ploppar det upp ännu ett fall dä en in...,2.0
2358,z02159_www_aftonbladet_se_debatt_article102554...,8,"Att verka för mångfald på arbetsplatser, i bos...",4.0
2807,z01878_www_svd_se_sverige-vet-inte-vad-invandr...,23,Medan de danska och norska forskarna är bekymr...,2.0


In [ ]:
#@title normalize 
import math

label = []

for idx, item in train_df.iterrows():
      lab = math.ceil(item.label)
      label.append(lab)

train_df["label_N"] = label

In [ ]:
train_df.sample(5)

,doc,par,text,label,label_N
1280,z03409_www_svd_se_sd-omprova-fri-vard-for-papp...,8,**Är det då till exempel **rimligt att en papp...,2.0,2
2305,z04032_www_svd_se_kd-lidingo-riksdagen-skyffla...,11,Vi representerar Lidingö som är en av Sveriges...,3.0,3
3293,z02516_www_aftonbladet_se_debatt_a_0aGyM_ungdo...,3,Under debatten på SVT opinion som jag deltog i...,3.0,3
3042,z02897_flashback-56535866,3,"Kostnaderna skenar överallt. Migrationsverket,...",3.0,3
305,z00063_flashback-47867942,1,Jonas Sjöstedt: Obligatoriskt flyktingmottagan...,3.0,3


In [ ]:
#@title sentence model
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, models

model_dir = 'Peltarion/xlm-roberta-longformer-base-4096'
mod = "KB/bert-base-swedish-cased"
sen_xlmr = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"

word_embedding_model = models.Transformer(mod, tokenizer_name_or_path= mod , max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 6.4 MB/s 
     |████████████████████████████████| 4.2 MB 36.9 MB/s 
     |████████████████████████████████| 1.2 MB 54.8 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 63.4 MB/s 
     |████████████████████████████████| 6.6 MB 54.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=241737a88a6bd324cf23fa04e1ca94454be292e44c403c945eccd9ca5b64834b
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/491 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at KB/bert-base-swedish-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/390k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

#Test

In [ ]:
#@title
#getting all test files

list_of_files = readDir("/content/split10_consecutive_average",split="test")

df_list = []

for item in list_of_files:
    df_list.append(pd.read_csv( item, delimiter = "\t"))

test_df = pd.concat(df_list,ignore_index=False)



In [ ]:
#@title
label = []

for idx, item in test_df.iterrows():
      lab = math.ceil(item.label)
      label.append(lab)

test_df["label_N"] = label

#EDA

In [ ]:
#@title
!git clone https://github.com/mosh98/swe_aug.git
! pip install -r /content/swe_aug/reqs.txt
!wget https://www.ida.liu.se/divisions/hcs/nlplab/swectors/swectors-300dim.txt.bz2
!bzip2 -dk /content/swectors-300dim.txt.bz2
word_vec_path = '/content/swectors-300dim.txt'
from swe_aug import EDA
aug = EDA.Enkel_Data_Augmentation(word_vec_path)

Cloning into 'swe_aug'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 118 (delta 61), reused 89 (delta 32), pack-reused 0
Receiving objects: 100% (118/118), 516.24 KiB | 10.54 MiB/s, done.
Resolving deltas: 100% (61/61), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 14.3 MB/s 
     |████████████████████████████████| 6.0 MB 54.0 MB/s 
     |████████████████████████████████| 24.2 MB 11.4 MB/s 
     |████████████████████████████████| 749 kB 48.8 MB/s 
     |████████████████████████████████| 42 kB 689 kB/s 
     |████████████████████████████████| 181 kB 58.8 MB/s 
     |████████████████████████████████| 457 kB 58.4 MB/s 
     |████████████████████████████████| 660 kB 47.4 MB/s 
     |████████████████████████████████| 10.1 MB 58.1 MB/s 
     |████████████████████████████████| 304 k

--2022-05-27 10:29:37--  https://www.ida.liu.se/divisions/hcs/nlplab/swectors/swectors-300dim.txt.bz2
Resolving www.ida.liu.se (www.ida.liu.se)... 130.236.57.131, 2001:6b0:17:2004::57:131
Connecting to www.ida.liu.se (www.ida.liu.se)|130.236.57.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 195655746 (187M) [application/x-bzip2]
Saving to: ‘swectors-300dim.txt.bz2’

swectors-300dim.txt 100%[===================>] 186.59M  55.9MB/s    in 3.5s    

2022-05-27 10:29:40 (53.6 MB/s) - ‘swectors-300dim.txt.bz2’ saved [195655746/195655746]

Downloaded pre-trained UDPipe model for 'sv' language
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Fraction of Dataset


In [ ]:
#@title
from sklearn.model_selection import train_test_split


def frac(dataframe, fraction,label):
    """Returns fraction of data"""
    if fraction == 1.0:
      return dataframe
      
    y = dataframe[label]
    train, test = train_test_split(dataframe,train_size=fraction,stratify = y)
    
    return train

def encode_df(dataframe, col = "text"):
  encoded = []
  for idx, item in dataframe.iterrows():
         encoded.append(model.encode(item[col])) 

  return encoded


reports = [] #clf_report (Before Augment, After Augment)


In [ ]:
#@title
test_embed = []

y_test = test_df.label_N

for idx, item in test_df.iterrows():
  test_embed.append(model.encode(item.text))
  

In [ ]:
#@title
from swe_aug.Other_Techniques import Type_SR
aug_t = Type_SR.type_DA(word_vec_path)

Already downloaded a model for the 'sv' language
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#@title
train_df

,doc,par,text,label,label_N
0,z01627_flashback-60850068,2,All invandring är inte dålig men få kan förnek...,2.0,2
1,z01627_flashback-60850068,3,Ser man tillbaka i tiden så var det bättre på ...,2.0,2
2,z01627_flashback-60850068,4,Många blundar för problemen eller försöker med...,2.0,2
3,z01627_flashback-60850068,5,"Vi hade dock krisen 2015, sanslöst ökat våld m...",1.0,1
4,z01627_flashback-60850068,6,Så frågan är när svenskarna kommer vakna? När ...,1.0,1
...,...,...,...,...,...
3869,z02107_flashback-15247839,4,"Nu var de, enligt dem själva, kristna också = ...",3.0,3
3870,z02514_flashback-54883701,1,Professor varnar: bostäderna snart slut mestad...,2.0,2
3871,z02514_flashback-54883701,2,I SvD varnar professor Stellan Lundström att e...,2.0,2
3872,z02514_flashback-54883701,3,Att det är invandringen som spetsar till läget...,2.0,2


In [ ]:

from sklearn.linear_model import SGDClassifier

split_percentage = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
label = "label_N"

for percentage in split_percentage:
    print("______________________________________________")
    
    print("Percentage",percentage)
    
    temp_train_df = frac(train_df, percentage/100,label )
    print("Before Augmentation size: ",temp_train_df.shape)

    y_train = temp_train_df.label_N
    
    train_embed = encode_df( temp_train_df )

    logreg = SGDClassifier(max_iter=3000)
    logreg.fit(train_embed, y_train)
    
    y_pred = logreg.predict(test_embed)
    
    r_1 = classification_report(y_test, y_pred)
    print("Before Augmentation")
    print(r_1)
    print(" ")
    
    #Augmentation EDA
    aug_samples = []
    for idx, item in temp_train_df.iterrows():
      txt = item.text
      lab = item.label_N
      list_of_augs = aug.enkel_augmentation(txt, alpha_sr=0.2, alpha_ri=0.2, alpha_rs=0.2, alpha_rd=0.2, num_aug=2)
      
      for element in list_of_augs:
          aug_samples.append({"text":element,"label_N":lab})

    
    new_aug_EDA_samples = pd.DataFrame(aug_samples)

    
    new_df = pd.concat([temp_train_df,new_aug_EDA_samples], ignore_index=True)
    
    print("After Augmentation size: ",new_df.shape[0])
    
    #augmented train test

    #encode
    train_embed_ = encode_df( new_df)
    
    y_train_ = new_df.label_N
    
    
    sgd_clf_2 = SGDClassifier(max_iter=3000)
    sgd_clf_2.fit(train_embed_, y_train_)

    y_pred = sgd_clf_2.predict(test_embed)


    r_2 = classification_report(y_test, y_pred)
    print("After Augmentation")
    print(r_2)

    #save the reports
    reports.append((percentage, r_1,r_2))



    print("______________________________________________")










______________________________________________
Percentage 10
Before Augmentation size:  (3864, 5)
Before Augmentation
              precision    recall  f1-score   support

           1       0.77      0.30      0.43       440
           2       0.63      0.45      0.53      1127
           3       0.64      0.81      0.71      2239
           4       0.56      0.63      0.59       849
           5       0.79      0.31      0.44       175

    accuracy                           0.63      4830
   macro avg       0.68      0.50      0.54      4830
weighted avg       0.64      0.63      0.61      4830

 
After Augmentation size:  19320
After Augmentation
              precision    recall  f1-score   support

           1       0.48      0.64      0.55       440
           2       0.61      0.48      0.53      1127
           3       0.66      0.81      0.73      2239
           4       0.72      0.43      0.54       849
           5       0.62      0.48      0.54       175

    accuracy  

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, models

#model_dir = 'Peltarion/xlm-roberta-longformer-base-4096'
#mod = "KB/bert-base-swedish-cased"
sen_xlmr = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"

word_embedding_model = models.Transformer(sen_xlmr, tokenizer_name_or_path= sen_xlmr , max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])



In [ ]:
    #Augmentation

aug_samples = []
original_augmented = [] #every element : orginal_text, (Tuple of augmented sentences)
for idx, item in train_df.iterrows():
      txt = item.text
      lab = item.label
      list_of_augs = aug.enkel_augmentation(txt, alpha_sr=0.2, alpha_ri=0.2, alpha_rs=0.2, alpha_rd=0.2, num_aug=2)
      
      temp_list = []
      for element in list_of_augs:
            aug_samples.append({"text":element,"label":lab})
            temp_list.append(element)
      
      original_augmented.append((txt,temp_list))

    

new_aug_samples = pd.DataFrame(aug_samples)

In [ ]:
from sentence_transformers import SentenceTransformer, util


def checkSimilarity(reference_string, list_of_strings, model):
    """
    This function takes a reference string and a list of strings and returns a list of strings that are similar to the reference string.

    :param reference_string:
    :param list_of_strings:
    :param model: sentence transformer model
    :return:
    """
    #1. Encode using Sentence Transformer
    #2. Calculate cosine similarity
    #3. Append the similirty value to the list
    # caclulcate percentage of elements below 0.9
    # return the list, and the percentage of elements below 0.9

    reference_encoded = model.encode(reference_string,convert_to_numpy=True) #encoded reference string
    list_of_cosine_similarity = []
    for string in list_of_strings:
        #print(string)
        encoded_string = model.encode(string,convert_to_numpy=True)

        #find coside similairity of enocded String and reference string
        similarity = util.cos_sim(reference_encoded, encoded_string)
        list_of_cosine_similarity.append(similarity.item())
    #print(list_of_cosine_similarity)
    #sum(i > 5 for i in j)
    percentage_of_elements_below_0_9 = sum(i < 0.9 for i in list_of_cosine_similarity) / len(list_of_cosine_similarity)

    return sum(i < 0.95 for i in list_of_cosine_similarity),percentage_of_elements_below_0_9

In [ ]:
low_semantic_sentences = []
for item in original_augmented:
  num, percentage = checkSimilarity(item[0],item[1],model)
  low_semantic_sentences.append(num)

In [ ]:
print("Original Sentences:",len(original_augmented))
print("Num of Augmented Sentences:",len(original_augmented)*4)


Original Sentences: 38640
Num of Augmented Sentences: 154560


In [ ]:
print("Num of Bad Augmented Sentences:",sum(low_semantic_sentences))

Num of Bad Augmented Sentences: 62422


In [ ]:
num_of_aug_samples = len(original_augmented)*4
print("Percentage of Bad Augmented Sentences:",(sum(low_semantic_sentences)/num_of_aug_samples)*100)

Percentage of Bad Augmented Sentences: 40.38690476190476
